In [502]:
#Aplicação para detecção de anomalias 

Para a construção de um modelo utilizando o Spark e MLlib são necessários 6 passos:

1) Construir e iniciar a seção SPARK
2) Implementar o carregamento dos dados para o spark: Carregar o arquivo, especificar o formato desejado e lê os dados como um Dataframe do Spark
3) Identificar as características a serem utilizadas para treinamento e teste do modelo
4) Instanciar as classes e os objetos dos algoritmos a serem utilizados
5) Utilizar o método fit() para realizar o treinamento do modelo
6) Avaliar o modelo

In [503]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [504]:
#inicia a seção para a utilização do spark
spark = SparkSession \
    .builder \
    .appName("DeteccaoAnomalias") \
    .getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [505]:
diretorioDataset="./temperature.csv"  #diretório que contém o arquivo a ser utilizado

In [506]:
data = spark.read.format("csv").options(header="true", inferschema="true").load(diretorioDataset)  #realiza a leitura do dataset

In [507]:
data.show(5,False)

+-------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----------------+-------------+-----+---------+---------+
|datetime           |Vancouver    |Portland     |San Francisco|Seattle      |Los Angeles  |San Diego    |Las Vegas    |Phoenix      |Albuquerque  |Denver       |San Antonio  |Dallas       |Houston      |Kansas City  |Minneapolis  |Saint Louis  |Chicago      |Nashville    |Indianapolis |Atlanta      |Detroit      |Jacksonville |Charlotte    |Miami        |Pittsburgh   |Toronto      |Philadelphia |New York     |Montreal     |Boston       |Beersheba    |Tel Aviv District|Eilat        

In [508]:
data.columns #mostra as colunas do dataset

['datetime',
 'Vancouver',
 'Portland',
 'San Francisco',
 'Seattle',
 'Los Angeles',
 'San Diego',
 'Las Vegas',
 'Phoenix',
 'Albuquerque',
 'Denver',
 'San Antonio',
 'Dallas',
 'Houston',
 'Kansas City',
 'Minneapolis',
 'Saint Louis',
 'Chicago',
 'Nashville',
 'Indianapolis',
 'Atlanta',
 'Detroit',
 'Jacksonville',
 'Charlotte',
 'Miami',
 'Pittsburgh',
 'Toronto',
 'Philadelphia',
 'New York',
 'Montreal',
 'Boston',
 'Beersheba',
 'Tel Aviv District',
 'Eilat',
 'Haifa',
 'Nahariyya',
 'Jerusalem']

In [509]:
data.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- Vancouver: double (nullable = true)
 |-- Portland: double (nullable = true)
 |-- San Francisco: double (nullable = true)
 |-- Seattle: double (nullable = true)
 |-- Los Angeles: double (nullable = true)
 |-- San Diego: double (nullable = true)
 |-- Las Vegas: double (nullable = true)
 |-- Phoenix: double (nullable = true)
 |-- Albuquerque: double (nullable = true)
 |-- Denver: double (nullable = true)
 |-- San Antonio: double (nullable = true)
 |-- Dallas: double (nullable = true)
 |-- Houston: double (nullable = true)
 |-- Kansas City: double (nullable = true)
 |-- Minneapolis: double (nullable = true)
 |-- Saint Louis: double (nullable = true)
 |-- Chicago: double (nullable = true)
 |-- Nashville: double (nullable = true)
 |-- Indianapolis: double (nullable = true)
 |-- Atlanta: double (nullable = true)
 |-- Detroit: double (nullable = true)
 |-- Jacksonville: double (nullable = true)
 |-- Charlotte: double (nullable = true)
 |-- M

In [510]:
data.count()  #conta a quantidade de registros

45253

In [511]:
#selecionando apenas o datatime e a coluna da cidade de Vancouver
dataAnalise=data.select('datetime','Vancouver')

In [512]:
dataAnalise.show(5)

+-------------------+-------------+
|           datetime|    Vancouver|
+-------------------+-------------+
|2012-10-01 12:00:00|         null|
|2012-10-01 13:00:00|       284.63|
|2012-10-01 14:00:00| 284.62904131|
|2012-10-01 15:00:00|284.626997923|
|2012-10-01 16:00:00|284.624954535|
+-------------------+-------------+
only showing top 5 rows



Tratando os dados

In [513]:
#filtrando apenas os dados que não possuem valores nulos 
from pyspark.sql.functions import col

dataNotNull=dataAnalise.filter(col('Vancouver').isNotNull())

In [514]:
dataNotNull.show(5)

+-------------------+-------------+
|           datetime|    Vancouver|
+-------------------+-------------+
|2012-10-01 13:00:00|       284.63|
|2012-10-01 14:00:00| 284.62904131|
|2012-10-01 15:00:00|284.626997923|
|2012-10-01 16:00:00|284.624954535|
|2012-10-01 17:00:00|284.622911147|
+-------------------+-------------+
only showing top 5 rows



In [515]:
from pyspark.sql.functions import monotonically_increasing_id  #biblioteca para a construção dos índices
df_plots = dataNotNull.withColumn("indice", monotonically_increasing_id())  #cria os indices para realizar o plot

In [516]:
df_plots.show(5)

+-------------------+-------------+------+
|           datetime|    Vancouver|indice|
+-------------------+-------------+------+
|2012-10-01 13:00:00|       284.63|     0|
|2012-10-01 14:00:00| 284.62904131|     1|
|2012-10-01 15:00:00|284.626997923|     2|
|2012-10-01 16:00:00|284.624954535|     3|
|2012-10-01 17:00:00|284.622911147|     4|
+-------------------+-------------+------+
only showing top 5 rows



In [517]:
#mostrando os dados
import pyspark.pandas as ps
import plotly

temp_df = ps.DataFrame(df_plots)
temp_df.plot.scatter(x='indice', y='Vancouver')



Aplicando o histograma para detecção de anomalias

In [518]:
#encontrando a média 
import numpy as np  #biblioteca utilizada para tratar vetores e matrizes
from pyspark.sql.functions import mean, stddev  #funções para encontrar a média e desvio padrão


In [519]:
list_stats = dataNotNull.select(mean(col('Vancouver')).alias('media'),stddev(col('Vancouver')).alias('desvioPadrao')).collect() #cria uma lista com os valores

In [520]:
media = list_stats[0]['media']
desvio = list_stats[0]['desvioPadrao']
print("Média: ", media)
print("Desvio Padrão: ", desvio)

Média:  283.86265374154124
Desvio Padrão:  6.640130616770929


In [521]:
df_stats = dataNotNull.select(mean(col('Vancouver')).alias('media'),stddev(col('Vancouver')).alias('desvioPadrao')) #cria o dataset com a média e o desvio padrão

In [522]:
df_stats.show()

+------------------+-----------------+
|             media|     desvioPadrao|
+------------------+-----------------+
|283.86265374154124|6.640130616770929|
+------------------+-----------------+



In [523]:
#utilizando a função describe
dataNotNull.describe().show()

+-------+------------------+
|summary|         Vancouver|
+-------+------------------+
|  count|             44458|
|   mean|283.86265374154124|
| stddev| 6.640130616770929|
|    min|            245.15|
|    max|             307.0|
+-------+------------------+



In [524]:
#definindo a funcao distancia
def distancia(x):
  media=283.8626
  desvio=6.6401
  return ((x - media)/desvio)

#definindo a funcao para verificar anomalias mais do que 2 desvios padrões (95% dos dados)
def anomalias(x):
  desvio=6.6401
  if (x > (2)):
    return 1
  else:
    return 0

#definindo as funções a serem utilizadas (registrando)
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import udf


distancia_udf_double = udf(lambda z: distancia(z), DoubleType())
anomalia_udf_int = udf(lambda z: anomalias(z), IntegerType())

In [525]:
data_new=dataNotNull.select('Vancouver',distancia_udf_double('Vancouver').alias('distancia'))
data_new.show()

+-------------+-------------------+
|    Vancouver|          distancia|
+-------------+-------------------+
|       284.63|0.11557054863631709|
| 284.62904131|0.11542616978660031|
|284.626997923|0.11511843541512777|
|284.624954535| 0.1148107008930567|
|284.622911147|0.11450296637099419|
|284.620867759| 0.1141952318489231|
|284.618824371|0.11388749732685204|
|284.616780983|0.11357976280478096|
|284.614737595| 0.1132720282827099|
|284.612694207|0.11296429376063881|
|284.610650819|0.11265655923856774|
|284.608607431|0.11234882471649667|
|284.606564043| 0.1120410901944256|
|284.604520655|0.11173335567235451|
|284.602477267|0.11142562115028344|
|284.600433879|0.11111788662821237|
|284.598390491| 0.1108101521061413|
|284.596347103|0.11050241758407878|
|284.594303716|0.11019468321260625|
|284.592260328|0.10988694869053518|
+-------------+-------------------+
only showing top 20 rows



In [526]:
from  pyspark.sql.functions import abs   #biblioteca necessária para cálculo do valor absoluto
data_new=data_new.select('Vancouver','distancia', abs(col('distancia')).alias("distanciaABS"))
data_new.show()

+-------------+-------------------+-------------------+
|    Vancouver|          distancia|       distanciaABS|
+-------------+-------------------+-------------------+
|       284.63|0.11557054863631709|0.11557054863631709|
| 284.62904131|0.11542616978660031|0.11542616978660031|
|284.626997923|0.11511843541512777|0.11511843541512777|
|284.624954535| 0.1148107008930567| 0.1148107008930567|
|284.622911147|0.11450296637099419|0.11450296637099419|
|284.620867759| 0.1141952318489231| 0.1141952318489231|
|284.618824371|0.11388749732685204|0.11388749732685204|
|284.616780983|0.11357976280478096|0.11357976280478096|
|284.614737595| 0.1132720282827099| 0.1132720282827099|
|284.612694207|0.11296429376063881|0.11296429376063881|
|284.610650819|0.11265655923856774|0.11265655923856774|
|284.608607431|0.11234882471649667|0.11234882471649667|
|284.606564043| 0.1120410901944256| 0.1120410901944256|
|284.604520655|0.11173335567235451|0.11173335567235451|
|284.602477267|0.11142562115028344|0.11142562115

In [527]:
data_new=data_new.select('Vancouver','distancia', "distanciaABS", anomalia_udf_int("distanciaABS").alias("isAnomaly"))
data_new.show()

+-------------+-------------------+-------------------+---------+
|    Vancouver|          distancia|       distanciaABS|isAnomaly|
+-------------+-------------------+-------------------+---------+
|       284.63|0.11557054863631709|0.11557054863631709|        0|
| 284.62904131|0.11542616978660031|0.11542616978660031|        0|
|284.626997923|0.11511843541512777|0.11511843541512777|        0|
|284.624954535| 0.1148107008930567| 0.1148107008930567|        0|
|284.622911147|0.11450296637099419|0.11450296637099419|        0|
|284.620867759| 0.1141952318489231| 0.1141952318489231|        0|
|284.618824371|0.11388749732685204|0.11388749732685204|        0|
|284.616780983|0.11357976280478096|0.11357976280478096|        0|
|284.614737595| 0.1132720282827099| 0.1132720282827099|        0|
|284.612694207|0.11296429376063881|0.11296429376063881|        0|
|284.610650819|0.11265655923856774|0.11265655923856774|        0|
|284.608607431|0.11234882471649667|0.11234882471649667|        0|
|284.60656

In [528]:
data_new.filter(col("isAnomaly")>0).show()

+---------+-------------------+------------------+---------+
|Vancouver|          distancia|      distanciaABS|isAnomaly|
+---------+-------------------+------------------+---------+
|   270.42|-2.0244574629900107|2.0244574629900107|        1|
|   269.81|-2.1163235493441337|2.1163235493441337|        1|
|   269.65| -2.140419571994399| 2.140419571994399|        1|
|   269.67|-2.1374075691631105|2.1374075691631105|        1|
|   269.67|-2.1374075691631105|2.1374075691631105|        1|
|   270.52|-2.0093974488336026|2.0093974488336026|        1|
|   269.15|-2.2157196427764654|2.2157196427764654|        1|
|   267.15|-2.5169199259047317|2.5169199259047317|        1|
|   266.15| -2.667520067468865| 2.667520067468865|        1|
|   266.15| -2.667520067468865| 2.667520067468865|        1|
|   266.15| -2.667520067468865| 2.667520067468865|        1|
|   266.15| -2.667520067468865| 2.667520067468865|        1|
|   266.15| -2.667520067468865| 2.667520067468865|        1|
|   265.15|-2.8181202090

In [529]:
#visualizando o histograma
# display(dataNotNull.select("Vancouver"))

temp_df = ps.DataFrame(dataNotNull.select("Vancouver"))
temp_df.plot.hist()

Boxplot

In [530]:
#boxplot
# display(dataNotNull.select("Vancouver"))

temp_df = ps.DataFrame(dataNotNull)
temp_df["Vancouver"].plot.box()


In [531]:
spark.sparkContext.parallelize( [np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([3.0, 30.0, 300.0])])  # an RDD of Vectors

ParallelCollectionRDD[1777] at readRDDFromFile at PythonRDD.scala:274

In [532]:
# Estatísticas com MLlib
from pyspark.mllib.stat import Statistics
coluna=dataNotNull.select("Vancouver")  #seleciona a coluna
coluna1= coluna.rdd.map(lambda x: [int (x[0])]) #aplica o map para transformar em vetor
estatistica=Statistics.colStats(coluna1) #aplica a estatística
print("Média: ",estatistica.mean())  # média
print("Variância: ", estatistica.variance())  # variância
print("Valores não nulos: ",estatistica.numNonzeros())  # numero de valores não zero

Média:  [283.36749291]
Variância:  [44.23613606]
Valores não nulos:  [44458.]


Aplicando o K-means

In [533]:
diretorioDataset="./worldcities.csv"  #diretório que contém o arquivo a ser utilizado

In [534]:
cities_df = spark.read.format("csv").options(header="true", inferschema="true").load(diretorioDataset)  #realiza a leitura do dataset

In [535]:
cities_df.printSchema()

root
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: double (nullable = true)
 |-- id: integer (nullable = true)



In [536]:
#mostrando o dataset
cities_df.show()

+--------------+--------------+--------+---------+-------------+----+----+--------------------+-------+-----------+----------+
|          city|    city_ascii|     lat|      lng|      country|iso2|iso3|          admin_name|capital| population|        id|
+--------------+--------------+--------+---------+-------------+----+----+--------------------+-------+-----------+----------+
|         Tokyo|         Tokyo|  35.685| 139.7514|        Japan|  JP| JPN|               Tōkyō|primary|   3.5676E7|1392685764|
|      New York|      New York| 40.6943| -73.9249|United States|  US| USA|            New York|   null|1.9354922E7|1840034016|
|   Mexico City|   Mexico City| 19.4424|  -99.131|       Mexico|  MX| MEX|    Ciudad de México|primary|   1.9028E7|1484247881|
|        Mumbai|        Mumbai|  19.017|   72.857|        India|  IN| IND|         Mahārāshtra|  admin|   1.8978E7|1356226629|
|     São Paulo|     Sao Paulo|-23.5587|  -46.625|       Brazil|  BR| BRA|           São Paulo|  admin|   1.884

In [537]:
#filtrando algumas cidades
cities_BR =cities_df.where(col("country")=="Brazil")
cities_BR.show(5)

+--------------+--------------+--------+--------+-------+----+----+-----------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|       admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-----------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|        São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|   Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|     Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA| Distrito Federal|primary| 3716996.0|1076144436|
+--------------+--------------+--------+--------+-------+----+----+-----

In [538]:
cities_MX =cities_df.where(col("country")=="Mexico")
cities_MX.show(5)

+-----------+-----------+-------+-------+-------+----+----+----------------+-------+----------+----------+
|       city| city_ascii|    lat|    lng|country|iso2|iso3|      admin_name|capital|population|        id|
+-----------+-----------+-------+-------+-------+----+----+----------------+-------+----------+----------+
|Mexico City|Mexico City|19.4424|-99.131| Mexico|  MX| MEX|Ciudad de México|primary|  1.9028E7|1484247881|
|Guadalajara|Guadalajara|  20.67|-103.33| Mexico|  MX| MEX|         Jalisco|  admin| 4198000.0|1484950208|
|  Monterrey|  Monterrey|  25.67|-100.33| Mexico|  MX| MEX|      Nuevo León|  admin| 3712000.0|1484559591|
|     Puebla|     Puebla|  19.05|  -98.2| Mexico|  MX| MEX|          Puebla|  admin| 2195000.0|1484416394|
|    Tijuana|    Tijuana|   32.5|-117.08| Mexico|  MX| MEX| Baja California|  minor| 1553000.0|1484708778|
+-----------+-----------+-------+-------+-------+----+----+----------------+-------+----------+----------+
only showing top 5 rows



In [539]:
cities_EUA =cities_df.where(col("country")=="United States")
cities_EUA.show(5)

+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
|       city| city_ascii|    lat|      lng|      country|iso2|iso3|admin_name|capital| population|        id|
+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
|   New York|   New York|40.6943| -73.9249|United States|  US| USA|  New York|   null|1.9354922E7|1840034016|
|Los Angeles|Los Angeles|34.1139|-118.4068|United States|  US| USA|California|   null|1.2815475E7|1840020491|
|    Chicago|    Chicago|41.8373| -87.6862|United States|  US| USA|  Illinois|   null|  8675982.0|1840000494|
|      Miami|      Miami|25.7839| -80.2102|United States|  US| USA|   Florida|   null|  6381966.0|1840015149|
|     Dallas|     Dallas|32.7936| -96.7662|United States|  US| USA|     Texas|   null|  5733259.0|1840019440|
+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
only showi

In [540]:
#criando um novo dataset através da função join
df_concat = cities_BR.union(cities_MX)
df_concat.show(5)

+--------------+--------------+--------+--------+-------+----+----+-----------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|       admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-----------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|        São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|   Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|     Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA| Distrito Federal|primary| 3716996.0|1076144436|
+--------------+--------------+--------+--------+-------+----+----+-----

In [541]:
#contando a quantidade de países diferentes
df_concat.groupby("country").count().show()

+-------+-----+
|country|count|
+-------+-----+
| Brazil|  387|
| Mexico|  188|
+-------+-----+



In [542]:
#adicionando cidades do Japão (nossas anomalias)
cities_JP =cities_df.where((col("city")=="Tokyo") | (col("city")=="Ōsaka"))
cities_JP.show()

+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+
| city|city_ascii|   lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|
+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+
|Tokyo|     Tokyo|35.685|139.7514|  Japan|  JP| JPN|     Tōkyō|primary|  3.5676E7|1392685764|
|Ōsaka|     Osaka| 34.75|135.4601|  Japan|  JP| JPN|     Ōsaka|  admin|  1.1294E7|1392419823|
+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+



In [543]:
#criando o dataset final
df_final = df_concat.union(cities_JP)

In [544]:
df_final.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA|   Distrito Federal|primary| 3716996.0|1076144436|
|        Recife|        Recife| -8.0756|-34.9156| Brazil

In [545]:
from pyspark.ml.evaluation import ClusteringEvaluator  #biblioteca utilizada para a avaliação em cada um dos clusters
from pyspark.ml.clustering import KMeans #biblioteca utilizada para a criação do modelo de clusterização utilizando o K-means

In [546]:
from pyspark.ml.feature import VectorAssembler  #transformando os dados em vetores de características

vecAssembler = VectorAssembler(inputCols=["lat","lng"], outputCol="features") #utilizada para transformar os dados em um vetor (define o objeto)
new_df = vecAssembler.transform(df_final) #Aplico a transformação
new_df.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|           features|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519| [-23.5587,-46.625]|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|  [-22.925,-43.225]|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|  [-19.915,-43.915]|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|     [-30.05,-51.2]|
|      Brasília|      Brasilia|-15

In [547]:
#aplica o processo de clusterização
kmeans = KMeans(k=3, seed=1)  # declara o objeto - 3 clusters 
model = kmeans.fit(new_df.select('features')) #aplica o treinamento

In [548]:
#cria o dataset com a indicação sobre qual cluster cada conjunto de dados foi adicionado
df_final = model.transform(new_df)
df_final.show() 

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|           features|prediction|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519| [-23.5587,-46.625]|         0|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|  [-22.925,-43.225]|         0|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|  [-19.915,-43.915]|         0|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|

In [549]:
df_final.groupby('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  387|
|         1|  188|
|         2|    2|
+----------+-----+



In [550]:
df_final.where(col("prediction")=='2').show()

+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+
| city|city_ascii|   lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|         features|prediction|
+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+
|Tokyo|     Tokyo|35.685|139.7514|  Japan|  JP| JPN|     Tōkyō|primary|  3.5676E7|1392685764|[35.685,139.7514]|         2|
|Ōsaka|     Osaka| 34.75|135.4601|  Japan|  JP| JPN|     Ōsaka|  admin|  1.1294E7|1392419823| [34.75,135.4601]|         2|
+-----+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+

